# Create a family knowledge base of the primitive relations: Parent(Parent, Child), Male(Person), Female(Person), and Married(Person, Person). Define a set of 10 relations (for example: Brother, Sister, Chacha, Chachi, Cousin...). Write a program to interactively answer the following kind of queries –“How is X related to Y”. How will you generate the shortest possible chain of relations? For example “Ramesh is Sunita’s maami’s cousin”.


In [1]:
relation = ['Brother','Dad','Mom','Sister','Cousin','Married','Uncle','Aunt','Grandfather','GrandMother']
print(len(relation))

10


In [32]:
def parse_family_relations_v3(file_path):
    relation_based_dict = {}
    
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():  # Ensure the line is not empty
                # Extracting the relation type and the individuals involved
                relation, people = line.strip().split('(')
                person1, person2 = people.strip(')').split(',')
                
                # Update the dictionary for the relation
                if relation not in relation_based_dict:
                    relation_based_dict[relation] = []
                relation_based_dict[relation].append([person1, person2])
                
                # Handle bidirectional relations such as 'Married'
                if relation == 'Married':
                    # For 'Married', we've already added both persons in the right order
                    # If other bidirectional relations need to be handled, similar logic could be applied here
                    pass
    
    return relation_based_dict

# Assuming the file path is 'family_graph.txt', which needs to be updated with the actual file path
file_path = 'family_graph.txt'  # Update this path
relations_dict = parse_family_relations_v3(file_path)
for relation, pairs in relations_dict.items():
    print(f"{relation}: {pairs}")


Mom: [['Alexis', 'Esha']]
Dad: [['Jayden', 'Esha'], ['Ram', 'Leo'], ['Ram', 'Jayden'], ['Leo', 'Jasmine']]
Brother: [['Michael', 'Esha']]
Sister: [['Esha', 'Michael']]
Married: [['Alexis', 'Jayden']]
Uncle: [['Leo', 'Michael'], ['Leo', 'Esha']]
Cousin: [['Esha', 'Jasmine']]


In [34]:
def find_ancestors_v2(person, relation_based_dict):
    ancestors = []
    
    def add_parents_of_sibling(sibling):
        """Add parents based on a sibling's known parents."""
        for relation in ['Mom', 'Dad']:
            if relation in relation_based_dict:
                for pair in relation_based_dict[relation]:
                    # If the sibling is found as a child in a parent relation
                    if sibling == pair[1]:
                        # Add this parent for the original person as well
                        if pair[0] not in ancestors:
                            ancestors.append(pair[0])
                            recurse_find(pair[0])

    def recurse_find(child):
        # Initially look for 'Mom' and 'Dad' relations directly
        direct_parent_found = False
        for relation in ['Mom', 'Dad']:
            if relation in relation_based_dict:
                for pair in relation_based_dict[relation]:
                    if child == pair[1] and pair[0] not in ancestors:
                        direct_parent_found = True
                        ancestors.append(pair[0])
                        recurse_find(pair[0])

        # If no direct parent is found, look for siblings as a way to find parents
        if not direct_parent_found:
            for pair in relation_based_dict.get('Brother', []) + relation_based_dict.get('Sister', []):
                if child == pair[0]:  # If the person is listed as a sibling
                    add_parents_of_sibling(pair[1])
                elif child == pair[1]:  # Or the sibling relation is listed the other way
                    add_parents_of_sibling(pair[0])

    recurse_find(person)
    return ancestors


# Test the updated function with an example
ancestors_of_michael = find_ancestors_v2('Jasmine', relations_dict)
print(f"Ancestors of Jasmine: {ancestors_of_michael}")


Ancestors of Jasmine: ['Leo', 'Ram']


In [35]:
def find_lca_v2(a, b, relation_based_dict):
    # Use the updated version of find_ancestors function
    ancestors_a = find_ancestors_v2(a, relation_based_dict)
    ancestors_b = find_ancestors_v2(b, relation_based_dict)
    
    # Find the first common ancestor in the list of ancestors for 'a' that also appears in 'b's ancestors
    # This relies on the ancestors being returned in order from closest to furthest
    lca = next((ancestor for ancestor in ancestors_a if ancestor in ancestors_b), None)
    return lca

# Assuming `relation_based_dict` is already defined and populated as shown previously
# Example use of the modified find_lca function
lca = find_lca_v2('Esha', 'Jasmine', relations_dict)
print(f"The lowest common ancestor (LCA) of Esha and Michael is: {lca}")


The lowest common ancestor (LCA) of Esha and Michael is: Ram


In [38]:
def calculate_relationship(person_a, person_b, relation_based_dict):
    # First, Check if their relationship already exists in the dictionary
    for relation, pairs in relation_based_dict.items():
        if [person_a, person_b] in pairs or [person_b, person_a] in pairs:
            return relation

    # Second, find the LCA for both individuals
    lca = find_lca_v2(person_a, person_b, relation_based_dict)
    if not lca:
        return "No common ancestors found, so no direct familial relationship."
    
    # Then, find the distance from each person to the LCA
    def find_distance_to_lca(person, lca, relation_based_dict):
        ancestors = find_ancestors_v2(person, relation_based_dict)
        distance = 0
        for ancestor in ancestors:
            distance += 1
            if ancestor == lca:
                return distance
        return -1  # In case the LCA isn't in the ancestors list, which shouldn't happen

    distance_a_to_lca = find_distance_to_lca(person_a, lca, relation_based_dict)
    distance_b_to_lca = find_distance_to_lca(person_b, lca, relation_based_dict)
    
    # Determine the relationship based on the distances
    if distance_a_to_lca == 1 and distance_b_to_lca == 1:
        return "Siblings"
    elif distance_a_to_lca == 1 or distance_b_to_lca == 1:
        return "Parent-Child" if distance_a_to_lca > distance_b_to_lca else "Child-Parent"
    elif distance_a_to_lca == 2 and distance_b_to_lca == 2:
        return "First Cousins"
    elif distance_a_to_lca > 2 or distance_b_to_lca > 2:
        return "Removed Cousins or further ancestor-descendant relationship"
    else:
        return "Uncategorized relationship"
relationship = calculate_relationship('Esha', 'Jasmine', relations_dict)
print(f"The relationship between Esha and Jasmine is: {relationship}")

The relationship between Esha and Jasmine is: Cousin
